In [53]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

# 모든 카테고리 다 긁어오기

In [62]:
url = requests.get('https://finance.naver.com/sise/sise_market_sum.naver?&page=1')
html = BeautifulSoup(url.text)

def make_table(html) :
    A = html.find('table', class_ = 'type_2')
    table = pd.read_html(str(A))[0]
    table = table.drop(columns=['N','토론실'])
    table = table.dropna(subset = '종목명')
    return table

In [78]:
items = html.find('table', class_ = 'item_list').find_all('input', attrs = {'name' : 'fieldIds'}) # 항목 선택표

items_box = []
for i in items :
    items_box.append(i['value'])
items_box

['quant',
 'ask_buy',
 'amount',
 'market_sum',
 'operating_profit',
 'per',
 'open_val',
 'ask_sell',
 'prev_quant',
 'property_total',
 'operating_profit_increasing_rate',
 'roe',
 'high_val',
 'buy_total',
 'frgn_rate',
 'debt_total',
 'net_income',
 'roa',
 'low_val',
 'sell_total',
 'listed_stock_cnt',
 'sales',
 'eps',
 'pbr',
 'sales_increasing_rate',
 'dividend',
 'reserve_ratio']

In [88]:
dic = {'menu' : 'market_sum',
      'returnUrl' : 'http://finance.naver.com/sise/sise_market_sum.naver?sosok=0&page=1',
      'fieldIds' : items_box}

url = requests.post('https://finance.naver.com/sise/field_submit.naver', data = dic) # 이제 정보요청을 보내는 것.
html = BeautifulSoup(url.text)
table = make_table(html)
table.to_excel('시가총액.xlsx', index = False)

In [90]:
# 코스피, 코스닥 모두 가져오고 (열도 모든 열) -> 하나의 데이터프레임

In [98]:
# 코스피 가져오기
url = requests.get('https://finance.naver.com/sise/sise_market_sum.naver')
html = BeautifulSoup(url.text)
page_num = int(html.find('td', class_ = 'pgRR').find('a')['href'].split('=')[-1])
page_num

47

In [102]:
tables = []
for i in range(1,page_num+1) :
    url = requests.post('https://finance.naver.com/sise/field_submit.naver', 
                        data = {'menu' : 'market_sum',
                      'returnUrl' : f'http://finance.naver.com/sise/sise_market_sum.naver?sosok=0&page={i}',
                      'fieldIds' : items_box})
    html = BeautifulSoup(url.text)
    tables.append(make_table(html))
kospi = pd.concat(tables)

,종목명,현재가,전일비,등락률,액면가,거래량,거래대금,전일거래량,시가,고가,...,주당순이익,보통주배당금,매출액증가율,영업이익증가율,외국인비율,PER,ROE,ROA,PBR,유보율
1,삼성전자,56600.0,하락 600,-1.05%,100.0,7955626.0,449473.0,14625181.0,57000.0,57100.0,...,4950.0,1446.0,16.20,398.34,50.07,11.43,9.03,7.10,0.98,41772.8
2,SK하이닉스,204500.0,"상승 4,000",+2.00%,5000.0,1342724.0,271648.0,3651613.0,199000.0,206000.0,...,14328.0,1200.0,-26.57,-213.52,55.84,14.27,-15.61,-8.95,2.16,1397.1
3,LG에너지솔루션,371500.0,"하락 2,000",-0.54%,500.0,58697.0,21829.0,140338.0,369500.0,376000.0,...,-1550.0,NaN,31.83,78.23,4.53,-239.68,6.36,3.91,4.30,16691.6
4,삼성바이오로직스,1159000.0,"하락 19,000",-1.61%,2500.0,23918.0,27781.0,108833.0,1181000.0,1187000.0,...,14788.0,NaN,23.10,13.22,13.58,78.37,9.12,5.26,7.79,5432.6
5,현대차,206000.0,"상승 3,500",+1.73%,5000.0,275311.0,56574.0,651155.0,203500.0,207000.0,...,45724.0,11400.0,14.43,53.96,37.18,4.51,13.68,4.56,0.54,6248.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9,TIGER 200 산업재,8240.0,하락 30,-0.36%,0.0,1030.0,8.0,17948.0,8270.0,8270.0,...,NaN,NaN,NaN,NaN,0.00,NaN,NaN,NaN,NaN,NaN
10,PLUS 코스닥150선물인버스,5440.0,하락 40,-0.73%,0.0,18.0,0.0,10576.0,5485.0,5485.0,...,NaN,NaN,NaN,NaN,0.00,NaN,NaN,NaN,NaN,NaN
11,HANARO 글로벌백신치료제MSCI,11790.0,상승 15,+0.13%,0.0,946.0,11.0,509.0,11795.0,11860.0,...,NaN,NaN,NaN,NaN,0.00,NaN,NaN,NaN,NaN,NaN
12,HANARO 미국애그테크,11750.0,상승 40,+0.34%,0.0,806.0,9.0,623.0,11555.0,11750.0,...,NaN,NaN,NaN,NaN,0.00,NaN,NaN,NaN,NaN,NaN


In [118]:
kospi = kospi.reset_index(drop = True)
kospi.to_excel('코스피 시총 모든 것.xlsx')
kospi

,종목명,현재가,전일비,등락률,액면가,거래량,거래대금,전일거래량,시가,고가,...,주당순이익,보통주배당금,매출액증가율,영업이익증가율,외국인비율,PER,ROE,ROA,PBR,유보율
0,삼성전자,56600.0,하락 600,-1.05%,100.0,7955626.0,449473.0,14625181.0,57000.0,57100.0,...,4950.0,1446.0,16.20,398.34,50.07,11.43,9.03,7.10,0.98,41772.8
1,SK하이닉스,204500.0,"상승 4,000",+2.00%,5000.0,1342724.0,271648.0,3651613.0,199000.0,206000.0,...,14328.0,1200.0,-26.57,-213.52,55.84,14.27,-15.61,-8.95,2.16,1397.1
2,LG에너지솔루션,371500.0,"하락 2,000",-0.54%,500.0,58697.0,21829.0,140338.0,369500.0,376000.0,...,-1550.0,NaN,31.83,78.23,4.53,-239.68,6.36,3.91,4.30,16691.6
3,삼성바이오로직스,1159000.0,"하락 19,000",-1.61%,2500.0,23918.0,27781.0,108833.0,1181000.0,1187000.0,...,14788.0,NaN,23.10,13.22,13.58,78.37,9.12,5.26,7.79,5432.6
4,현대차,206000.0,"상승 3,500",+1.73%,5000.0,275311.0,56574.0,651155.0,203500.0,207000.0,...,45724.0,11400.0,14.43,53.96,37.18,4.51,13.68,4.56,0.54,6248.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2305,TIGER 200 산업재,8240.0,하락 30,-0.36%,0.0,1030.0,8.0,17948.0,8270.0,8270.0,...,NaN,NaN,NaN,NaN,0.00,NaN,NaN,NaN,NaN,NaN
2306,PLUS 코스닥150선물인버스,5440.0,하락 40,-0.73%,0.0,18.0,0.0,10576.0,5485.0,5485.0,...,NaN,NaN,NaN,NaN,0.00,NaN,NaN,NaN,NaN,NaN
2307,HANARO 글로벌백신치료제MSCI,11790.0,상승 15,+0.13%,0.0,946.0,11.0,509.0,11795.0,11860.0,...,NaN,NaN,NaN,NaN,0.00,NaN,NaN,NaN,NaN,NaN
2308,HANARO 미국애그테크,11750.0,상승 40,+0.34%,0.0,806.0,9.0,623.0,11555.0,11750.0,...,NaN,NaN,NaN,NaN,0.00,NaN,NaN,NaN,NaN,NaN


In [108]:
# 코스닥 가져오기
url = requests.get('https://finance.naver.com/sise/sise_market_sum.naver?sosok=1&page=1')
html = BeautifulSoup(url.text)
page_num = int(html.find('td', class_ = 'pgRR').find('a')['href'].split('=')[-1])
page_num

36

In [110]:
tables = []
for i in range(1,page_num+1) :
    url = requests.post('https://finance.naver.com/sise/field_submit.naver', 
                        data = {'menu' : 'market_sum',
                      'returnUrl' : f'http://finance.naver.com/sise/sise_market_sum.naver?sosok=1&page={i}',
                      'fieldIds' : items_box})
    html = BeautifulSoup(url.text)
    tables.append(make_table(html))
kosdaq = pd.concat(tables)
kosdaq = kosdaq.reset_index(drop = True)
kosdaq

,종목명,현재가,전일비,등락률,액면가,거래량,거래대금,전일거래량,시가,고가,...,주당순이익,보통주배당금,매출액증가율,영업이익증가율,외국인비율,PER,ROE,ROA,PBR,유보율
0,알테오젠,395000.0,"상승 1,500",+0.38%,500.0,173465.0,67829.0,416909.0,397000.0,397000.0,...,-55.0,0.0,235.08,66.85,16.01,-7181.82,-2.27,-1.43,118.32,445.0
1,에코프로비엠,137900.0,"상승 4,800",+3.61%,500.0,376563.0,51215.0,715115.0,133000.0,138200.0,...,-2222.0,NaN,28.81,-59.01,10.48,-62.06,-0.64,1.41,10.39,2633.7
2,HLB,87800.0,상승 100,+0.11%,500.0,289156.0,25336.0,1322836.0,87800.0,88200.0,...,-1424.0,0.0,-76.13,-67.45,22.72,-61.66,-32.55,-23.97,21.95,1058.0
3,에코프로,66800.0,"상승 1,400",+2.14%,100.0,452556.0,29978.0,977768.0,64500.0,67200.0,...,-1365.0,0.0,28.73,-51.38,20.03,-48.94,3.43,2.10,5.82,9676.2
4,레인보우로보틱스,390500.0,"상승 3,500",+0.90%,500.0,119698.0,46642.0,412248.0,394000.0,395000.0,...,-1055.0,NaN,12.07,-3527.67,8.35,-370.14,-0.93,-0.81,58.49,1223.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1786,투비소프트,584.0,하락 27,-4.42%,500.0,173651.0,102.0,816914.0,611.0,629.0,...,-3132.0,0.0,-0.46,8.06,2.89,-0.19,-41.73,-17.86,0.20,-8.0
1787,셀리버리,117.0,하락 26,-18.18%,500.0,5918223.0,694.0,31418566.0,125.0,125.0,...,-166.0,0.0,-18.55,70.75,3.78,-0.70,472.60,-59.26,-0.22,-289.3
1788,CNH,109.0,보합0,0.00%,2500.0,0.0,0.0,0.0,0.0,0.0,...,-2570.0,0.0,-17.10,76.64,6.07,-0.04,-18.80,-3.89,0.36,5.8
1789,소프트센우,7000.0,상승 80,+1.16%,200.0,1185.0,8.0,204.0,7050.0,7050.0,...,-64.0,NaN,NaN,NaN,0.02,-109.38,NaN,NaN,13.60,NaN


In [112]:
kosdaq.to_excel('코스닥 시총 모든 것.xlsx')

In [122]:
total = pd.concat([kospi,kosdaq]).reset_index(drop=True)
total

,종목명,현재가,전일비,등락률,액면가,거래량,거래대금,전일거래량,시가,고가,...,주당순이익,보통주배당금,매출액증가율,영업이익증가율,외국인비율,PER,ROE,ROA,PBR,유보율
0,삼성전자,56600.0,하락 600,-1.05%,100.0,7955626.0,449473.0,14625181.0,57000.0,57100.0,...,4950.0,1446.0,16.20,398.34,50.07,11.43,9.03,7.10,0.98,41772.8
1,SK하이닉스,204500.0,"상승 4,000",+2.00%,5000.0,1342724.0,271648.0,3651613.0,199000.0,206000.0,...,14328.0,1200.0,-26.57,-213.52,55.84,14.27,-15.61,-8.95,2.16,1397.1
2,LG에너지솔루션,371500.0,"하락 2,000",-0.54%,500.0,58697.0,21829.0,140338.0,369500.0,376000.0,...,-1550.0,NaN,31.83,78.23,4.53,-239.68,6.36,3.91,4.30,16691.6
3,삼성바이오로직스,1159000.0,"하락 19,000",-1.61%,2500.0,23918.0,27781.0,108833.0,1181000.0,1187000.0,...,14788.0,NaN,23.10,13.22,13.58,78.37,9.12,5.26,7.79,5432.6
4,현대차,206000.0,"상승 3,500",+1.73%,5000.0,275311.0,56574.0,651155.0,203500.0,207000.0,...,45724.0,11400.0,14.43,53.96,37.18,4.51,13.68,4.56,0.54,6248.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4096,투비소프트,584.0,하락 27,-4.42%,500.0,173651.0,102.0,816914.0,611.0,629.0,...,-3132.0,0.0,-0.46,8.06,2.89,-0.19,-41.73,-17.86,0.20,-8.0
4097,셀리버리,117.0,하락 26,-18.18%,500.0,5918223.0,694.0,31418566.0,125.0,125.0,...,-166.0,0.0,-18.55,70.75,3.78,-0.70,472.60,-59.26,-0.22,-289.3
4098,CNH,109.0,보합0,0.00%,2500.0,0.0,0.0,0.0,0.0,0.0,...,-2570.0,0.0,-17.10,76.64,6.07,-0.04,-18.80,-3.89,0.36,5.8
4099,소프트센우,7000.0,상승 80,+1.16%,200.0,1185.0,8.0,204.0,7050.0,7050.0,...,-64.0,NaN,NaN,NaN,0.02,-109.38,NaN,NaN,13.60,NaN


In [126]:
total.to_excel('코스피 + 코스닥 시총 모든 것.xlsx')

In [129]:
# !pip install undetected-chromedriver


Defaulting to user installation because normal site-packages is not writeable
